In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
#библиотека для регулярных выражений
import re 


In [144]:
data = pd.read_csv('/home/mike/Documents/Coding/Data/wine_cleared.csv')
display(data.head())

,Unnamed: 0,country,description,designation,points,price,province,region_1,taster_name,taster_twitter_handle,title,variety,winery
0,0,Italy,"Aromas include tropical fruit, broom, brimston...",Vulkà Bianco,87,35.363389,Sicily & Sardinia,Etna,Kerin O’Keefe,@kerinokeefe,Nicosia 2013 Vulkà Bianco (Etna),White Blend,Nicosia
1,1,Portugal,"This is ripe and fruity, a wine that is smooth...",Avidagos,87,15.000000,Douro,unknown,Roger Voss,@vossroger,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,Quinta dos Avidagos
2,2,US,"Tart and snappy, the flavors of lime flesh and...",unknown,87,14.000000,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,Rainstorm
3,3,US,"Pineapple rind, lemon pith and orange blossom ...",Reserve Late Harvest,87,13.000000,Michigan,Lake Michigan Shore,Alexander Peartree,unknown,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,St. Julian
4,4,US,"Much like the regular bottling from 2012, this...",Vintner's Reserve Wild Child Block,87,65.000000,Oregon,Willamette Valley,Paul Gregutt,@paulgwine,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,Sweet Cheeks


In [145]:
# для удобства сразу преобразуем признак в int
data['price_round'] = data['price'].round().astype(int)

# РАЗБОР ТЕКСТА

In [146]:
regex = '\d+' # регулярное выражение для нахождения чисел
data['year'] = data['title'].str.findall(regex).str.get(0)

# РАЗБОР КАТЕГОРИЙ

In [147]:
data['is_usa'] = data['country'].apply(lambda x: 1 if x == 'US' else 0)
data['is_france'] = data['country'].apply(lambda x: 1 if x == 'France' else 0)
data['is_italy'] = data['country'].apply(lambda x: 1 if x == 'Italy' else 0)

In [148]:
# преобразуем признак year в объект datetime для удобного сравнения дат
data['year'] = pd.to_datetime(data['year'], errors='coerce')

# для сравнения используем год, заполняем значения признака old_wine, где год вина меньше 2010
data['old_wine'] = data['year'].apply(lambda x: 1 if x.year < 2010 else 0) 
data['old_wine'].sum()

39594

In [149]:
regex = '\((.*?)\)'
data['locality'] = data['title'].str.findall(regex).str.get(0)

# РАБОТА С ФАЙЛАМИ

In [150]:
country_population = pd.read_csv('/home/mike/Documents/Coding/Data/country_population.csv', sep=';')
country_population

,country,population
0,China,"1,411,778,724"
1,India,"1,386,584,581"
2,US,"333,022,386"
3,Indonesia,"271,350,000"
4,Pakistan,"225,200,000"
...,...,...
236,Niue,"1,549"
237,Tokelau,"1,501"
238,Vatican City,825
239,Cocos Islands,573


In [151]:
data = data.join(country_population.set_index('country'), on='country')

In [152]:
area_country = pd.read_csv('/home/mike/Documents/Coding/Data/country_area.csv', sep=';')
area_country

,country,area
0,Russia,17075400.00
1,Canada,9984670.00
2,China,9596960.00
3,US,9372610.00
4,Brazil,8547000.00
...,...,...
188,San Marino,61.00
189,Tuvalu,26.00
190,Nauru,21.00
191,Monaco,2.00


In [153]:
data = data.join(area_country.set_index('country'), on='country')

In [154]:
data['year'] = pd.to_datetime(data['year'], errors='coerce')
data['years_diff'] = (pd.to_datetime('2022-01-12') - data['year']).dt.days
data['years_diff'].max()

98627.0

# Кодирование признаков

In [155]:
pip install category_encoders

Note: you may need to restart the kernel to use updated packages.


In [156]:
import category_encoders as ce

In [157]:
# Закодируйте признак year в датасете винных обзоров порядковым кодированием.

ord_encoder = ce.OrdinalEncoder(cols=['year'])
year_col = ord_encoder.fit_transform(data['year'])
data = pd.concat([data, year_col], axis=1)

In [158]:
#признак, обозначающий имя сомелье (taster_name), является номинальным. Закодируйте его, используя изученные ранее способы кодирования.

encoder = ce.OneHotEncoder(cols=['taster_name'])
type_bin = encoder.fit_transform(data['taster_name'])
data = pd.concat([data, type_bin], axis=1)

In [159]:
bin_encoder = ce.BinaryEncoder(cols=['country'])
type_bin = bin_encoder.fit_transform(data['country'])
data = pd.concat([data, type_bin], axis=1)

In [160]:
#определите подходящий способ кодирования признака taster_twitter_handle из датасета винных обзоров и закодируйте его.

encoder = ce.BinaryEncoder(cols=['taster_twitter_handle'])
type_bin = encoder.fit_transform(data['taster_twitter_handle'])
data = pd.concat([data, type_bin], axis=1)

# Нормализация. Стандартизация

In [161]:
from sklearn import preprocessing

In [162]:
#Нормализуйте признак price. Используйте подходящий тип нормализации.


#r_scaler = preprocessing.RobustScaler()

#data_r = r_scaler.fit_transform(pd.DataFrame(data = data['price']))
#round(data_r.sum(),)

In [197]:
data['price'][129968]

30.0

In [198]:
#Стандартизируйте исходный признак price.

s_scaler = preprocessing.StandardScaler()
data_s = s_scaler.fit_transform(pd.DataFrame(data = data['price']))

In [199]:
data['price_standartized'] = data_s

In [200]:
data['price_standartized'][129968]

-0.13551516993245472

In [201]:
# Построим график корреляции всех величин. Для простоты воспользуемся корреляцией Пирсона.

# работа с визуализацией
import matplotlib.pyplot as plt
import seaborn as sns

fig, ax = plt.subplots(figsize=(15,10))
sns.heatmap(data.corr(), annot=True, linewidths=.5, ax=ax)

# Удалим самые сильно скоррелированные пары
data = data.drop(['is_usa', 'is_france', 'is_italy', 'price_round', 'area'], axis=1)

# Проверяем, что сильно скоррелированных признаков не осталось
#sns.heatmap(data.corr(), annot=True, linewidths=.5, ax=ax)

KeyError: "['is_usa' 'is_france' 'is_italy' 'price_round' 'area'] not found in axis"

Error in callback <function flush_figures at 0x7fb44c249ee0> (for post_execute):


KeyboardInterrupt: 

# Проверка

In [202]:
def set_avg_trestbps(x):
    if x['sex'] == 1:
        if x['age'] == 20:
            return 123
        if 20 < x['age'] <= 30:
            return 126
        if 30 < x['age'] <= 40:
            return 129
        if 40 < x['age'] <= 50:
            return 135
        if 50 < x['age'] <= 60:
            return 142
        if x['age'] > 60:
            return 142
    if x['sex'] == 0:
        if x['age'] == 20:
            return 116
        if 20 < x['age'] <= 30:
            return 120
        if 30 < x['age'] <= 40:
            return 127
        if 40 < x['age'] <= 50:
            return 137
        if 50 < x['age'] <= 60:
            return 144
        if x['age'] > 60:
            return 169

data['trestbps_mean'] = data.apply(set_avg_trestbps, axis=1)

KeyError: 'sex'